In [5]:
import pandas as pd
import numpy as np
import matplotlib
import plotnine as ggplot
from plotnine import *
%matplotlib inline

import glob
import re

import time

from pandas import DataFrame
from IPython.display import HTML

import sys
sys.path.append("fastai/old/")
from fastai_v7 import structured

### Start

In [3]:
!ls /home/maghoi/bin/software/PDB_Tool

1pazA.txt   PDB_Tool	 test.acc	 test.cle	 test.pdb
examples    README	 test.acc_value  test.distances  test.sse
LICENCE.md  source_code  test.ami	 test.feature


- Want AMI, SSE, ACC and FEAT = 13

In [3]:
1+3+4+5

13

In [4]:
!/home/maghoi/bin/software/PDB_Tool/PDB_Tool

========================================================|
PDB_Tool  (version 4.80) [2018.05.20]                   |
          a versatile tool to process PDB file          |
Usage:   ./PDB_Tool <-i input> <-r range> <-o output>   |
Or,      ./PDB_Tool <-i inroot> <-L list> <-o outroot>  |
--------------------------------------------------------|
For <-i input> input type,                              |
-i input  : input original PDB file, e.g., 1col.pdb     |
-r range  : input residue range, e.g., A:1-51           |
-o output : output PDB file, e.g., 1colA.pdb            |
--------------------------------------------------------|
For <-L list> input type,                               |
-i inroot  : input directory of original PDBs           |
-o outroot : output directory for generated PDBs        |
The rows in <list> indicate [input range output]        |
          e.g.,  1col.pdb A:1-51 1colA.pdb              |
========================================================|
              

In [67]:
%%bash
#ls -al /scratch/maghoi/pMHC_data/features
#cd /scratch/maghoi/pMHC_data/features
#rm /scratch/maghoi/pMHC_data/features/*

### PDB_Tool Feature extraction (complex + peptide)

In [6]:
start = time.time()

#### Run PDB_tool

In [163]:
%%bash
cd /scratch/maghoi/pMHC_data/model_structures
#Extracting features for full complex
find . -name "*pMHC-TCR.pdb" -exec /home/maghoi/bin/software/PDB_Tool/PDB_Tool -i {} -o /scratch/maghoi/pMHC_data/features/{} -F 4 -r ! \;

#Extracting peptide amino acid sequence
find . -name "*pMHC-TCR.pdb" -exec /home/maghoi/bin/software/PDB_Tool/PDB_Tool -i {} -o /scratch/maghoi/pMHC_data/features/{}_pep -F 1 -r P:@ \;


#### Check outdir

In [173]:
%%bash
ls /scratch/maghoi/pMHC_data/features/ |head

1g6r_1p_N1_1t_80s_pMHC-TCR.ami
1g6r_1p_N1_1t_80s_pMHC-TCR.cle
1g6r_1p_N1_1t_80s_pMHC-TCR.feature
1g6r_1p_N1_1t_80s_pMHC-TCR.pdb
1g6r_1p_N1_1t_80s_pMHC-TCR.sse
1g6r_1p_N1_1t_80s_pMHC-TCR_pep.pdb
1g6r_1p_N1_1t_90s_pMHC-TCR.ami
1g6r_1p_N1_1t_90s_pMHC-TCR.cle
1g6r_1p_N1_1t_90s_pMHC-TCR.feature
1g6r_1p_N1_1t_90s_pMHC-TCR.pdb


In [165]:
print("Time:", round(time.time() - start, 2), "s")

Time: 122.0 s


#### Renaming filename.pdb_pep to filename_pep.pdb

In [171]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
find . -type f -name "*_pep" |
    while IFS= read file_name; do
        mv $file_name ${file_name:0:-8}_pep.pdb
    done

In [172]:
!ls /scratch/maghoi/pMHC_data/features/ |head

1g6r_1p_N1_1t_80s_pMHC-TCR.ami
1g6r_1p_N1_1t_80s_pMHC-TCR.cle
1g6r_1p_N1_1t_80s_pMHC-TCR.feature
1g6r_1p_N1_1t_80s_pMHC-TCR.pdb
1g6r_1p_N1_1t_80s_pMHC-TCR.sse
1g6r_1p_N1_1t_80s_pMHC-TCR_pep.pdb
1g6r_1p_N1_1t_90s_pMHC-TCR.ami
1g6r_1p_N1_1t_90s_pMHC-TCR.cle
1g6r_1p_N1_1t_90s_pMHC-TCR.feature
1g6r_1p_N1_1t_90s_pMHC-TCR.pdb
ls: write error: Broken pipe


#### Check that there is 1464 feature files and peptide files
#### (will extract pep seq using pdb2pep)

In [5]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
ls *pMHC-TCR.feature*|wc
ls *pMHC-TCR_pep*|wc

   1464    1464   51240
   1464    1464   51240


bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


### Convert feature files to tab-delimited, extract relevant columns

In [296]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
mkdir -p /scratch/maghoi/pMHC_data/features2/
find . -type f -name "*pMHC-TCR.feature" |
    while IFS= read file_name; do
        #cat $file_name
        sed -r 's/ +/\t/g' $file_name | cut -f 2,3,5,6,7,8,9,10 > ../features2/$file_name
    done

In [301]:
%%bash
ls /scratch/maghoi/pMHC_data/features2/|wc
head /scratch/maghoi/pMHC_data/features2/1g6r_1p_N1_1t_80s_pMHC-TCR.feature

   1464    1464   51240
Num	Res	SSE	CLE	ACC	pACC	CNa	CNb
1	G	L	R	2	99	2	2
2	P	L	R	2	70	4	5
3	H	E	E	1	29	6	8
4	S	E	F	1	12	7	9
5	L	E	D	0	0	9	9
6	R	E	E	1	26	8	7
7	Y	E	E	0	0	8	10
8	F	E	E	1	31	7	8
9	V	E	E	0	2	7	13


In [68]:
#!ls -al /scratch/maghoi/pMHC_data/features2/

In [13]:
pd.read_csv("/scratch/maghoi/pMHC_data/features2/1g6r_1p_N1_1t_80s_pMHC-TCR.feature", delimiter="\t")[180:190]


,Num,Res,SSE,CLE,ACC,pACC,CNa,CNb
180,181,N,L,R,0,0,5,10
181,182,I,L,C,0,0,3,7
182,183,R,L,C,0,2,7,7
183,184,Y,L,C,0,0,4,6
184,185,V,L,D,0,1,5,8
185,186,K,L,C,0,1,7,11
186,187,V,L,R,0,3,6,7
187,188,Q,L,R,1,31,6,9
188,189,S,E,R,2,53,6,4
189,190,V,E,E,0,10,11,11


In [12]:
pd.read_csv("/scratch/maghoi/pMHC_data/features2/2oi9_3p_S1_1t_99s_pMHC-TCR.feature", delimiter="\t")[180:190]


,Num,Res,SSE,CLE,ACC,pACC,CNa,CNb
180,181,P,L,C,0,0,1,2
181,182,F,L,C,0,1,4,7
182,183,D,L,C,0,8,7,9
183,184,L,L,R,0,1,4,8
184,185,L,L,R,2,52,4,6
185,186,S,E,R,2,67,4,3
186,187,V,E,E,1,11,8,8
187,188,R,E,D,2,69,4,4
188,189,E,T,C,2,43,7,6
189,190,G,T,N,2,53,4,4


### Extract peptide seqs

In [11]:
%%bash
cd /scratch/maghoi/pMHC_data/features/
find . -type f -name "*pMHC-TCR_pep.pdb" |
    while IFS= read file_name; do
        /home/maghoi/bin/software/pdb2fasta/pdb2fasta.sh $file_name > ../pep/$file_name.fasta
    done

In [10]:
!cat /scratch/maghoi/pMHC_data/pep/1g6r_1p_N1_1t_90s_pMHC-TCR_pep.pdb.fasta

>./1g6r_1p_N1_1t_90s_pMHC-TCR_pep | chain P | 8 aa
KNIRYVKV


### Save to csv

In [ ]:
import glob
filelist = glob.glob("/scratch/maghoi/pMHC_data/features2/*")

In [1]:
%timeit
import os
for i in range(0, len(filelist)):
    df_raw = pd.read_csv(filelist[i], delimiter = "\t")
    #df = structured.proc_df(df_raw); df = df[0]
    df = df_raw
    filename = os.path.basename(filelist[i])
    df.to_csv("/scratch/maghoi/pMHC_data/features3/" + filename + ".csv")
    print(filename)
    print(df.shape)

NameError: name 'filelist' is not defined

### Data generator (use this 27/03/19)

In [69]:
#!ls /scratch/maghoi/pMHC_data/features3/
#!cat /scratch/maghoi/pMHC_data/features3/3vxm_3p_P1_1t_80s_pMHC-TCR.feature.csv

In [3]:
#Start position 0
def data_generator(ix_train = 1080, ix_val = 256, ix_test = 128, data_size=1464):
        filelist = glob.glob("/home/maghoi/pMHC_data/features3/*.csv")
        
        print(len(filelist))
        
        for i in range(len(filelist)):
            r = re.compile(r'.*/features3/(.*).feature')
            m = r.match(filelist[i])
            filename = m.group(1)
            filenames.append(filename)
        
        #Load data into dfs
        filelist_loaded = []
        for i in range(0, len(filelist)):
            df = pd.read_csv(filelist[i])
            #df = df.drop(["Unnamed: 0", "Num", "pACC", "CNb"], axis = 1)
            df = df.drop(["Unnamed: 0", "Num"], axis = 1)
            
            #Convert categorical values (res + solvent acc cateogry) to string
            filelist_loaded.append(df)
        return filelist_loaded, filenames

- Load feature files into filelist_loded and filenames

In [4]:
filelist_loaded = []
filenames = []
filelist_loaded, filenames = data_generator()
filenames = pd.DataFrame(filenames)

1464


- Extract aligned sequences from All_identity_xlsx
- Fix names so they are identical

In [20]:
aligned = pd.read_excel("/home/maghoi/pMHC_data/All_identity.xlsx")
aligned = aligned.iloc[:, [0, 3]]
aligned.Name = aligned.Name + "_pMHC-TCR"

In [23]:
#aligned.loc[aligned.Name == filenames[2]].Aligned_sequence

In [14]:
for i in range(len(filenames)):
    print(aligned.loc[aligned.Name == filenames[i]])
    print("Items:", i)

                           Name  \
224  2p5e_2p_N2_1t_95s_pMHC-TCR   
248  2p5w_2p_N2_1t_95s_pMHC-TCR   
272  2pye_2p_N2_1t_95s_pMHC-TCR   
296  2vlk_4p_N2_1t_95s_pMHC-TCR   
320  2vlr_4p_N2_1t_95s_pMHC-TCR   
344  2ypl_3p_N2_1t_95s_pMHC-TCR   
368  3e3q_3p_N2_1t_95s_pMHC-TCR   
392  3ffc_4p_N2_1t_95s_pMHC-TCR   
416  3gsn_0p_N2_1t_95s_pMHC-TCR   
440  3hg1_0p_N2_1t_95s_pMHC-TCR   
464  3mv7_4p_N2_1t_95s_pMHC-TCR   
488  3mv8_4p_N2_1t_95s_pMHC-TCR   
512  3mv9_4p_N2_1t_95s_pMHC-TCR   
536  3qdg_0p_N2_1t_95s_pMHC-TCR   
560  3qdj_2p_N2_1t_95s_pMHC-TCR   
584  3qdm_0p_N2_1t_95s_pMHC-TCR   
608  3qeq_2p_N2_1t_95s_pMHC-TCR   
632  3tfk_3p_N2_1t_95s_pMHC-TCR   
656  3tjh_4p_N2_1t_95s_pMHC-TCR   

                                      Aligned_sequence  
224  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
248  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
272  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
296  GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...  
320  GSHSMRYFF

KeyError: 1

Check that filenames and aligned names match

In [24]:
len(filenames), len(aligned)

(1464, 1464)

### Align raw sequences to pre-aligned sequences
### Extract features from .feature
### Add new features for MHC, pep, TCR
- Pre-aligned seqs in All_identity.xlsx
- Residue to residue comparison, replace missing (padded) with -1
- Add features for corresponding positions, with -1 for missing (padded) positions
- Add to dfs (list of dataframes)

In [25]:
pd.DataFrame(filelist_loaded[0]).describe()

,ACC,pACC,CNa,CNb
count,411.000000,411.000000,411.000000,411.000000
mean,0.922141,29.209246,5.401460,5.785888
std,0.795580,27.248361,2.649786,2.962842
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.000000,4.000000,3.000000
50%,1.000000,24.000000,5.000000,6.000000
75%,2.000000,45.500000,8.000000,8.000000
max,2.000000,100.000000,12.000000,13.000000


In [26]:
filelist_loaded[0].iloc[:]

,Res,SSE,CLE,ACC,pACC,CNa,CNb
0,G,L,R,2,86,3,2
1,S,L,R,2,71,5,5
2,H,E,E,1,17,7,9
3,S,E,E,1,19,7,8
4,M,E,D,0,0,9,10
5,R,E,E,1,27,8,6
6,Y,E,D,0,0,8,10
7,F,E,E,1,38,7,7
8,F,E,E,0,1,7,9
9,T,E,E,1,29,8,7


In [32]:
len(aligned.loc[aligned.Name == filenames[0]].Aligned_sequence.values[0])

468

In [13]:
start = time.time()
dfs = []

for x in range(len(filenames)):
    #Sequences
    aligned_seq = aligned.loc[aligned.Name == filenames[x]].Aligned_sequence.values[0]
    raw_seq = filelist_loaded[x].iloc[:, 0].values
    
    #Raw features
    raw_1 = filelist_loaded[x].iloc[:, 1].values
    raw_2 = filelist_loaded[x].iloc[:, 2].values
    raw_3 = filelist_loaded[x].iloc[:, 3].values
    raw_4 = filelist_loaded[x].iloc[:, 4].values
    raw_5 = filelist_loaded[x].iloc[:, 5].values
    raw_6 = filelist_loaded[x].iloc[:, 6].values

    #New features
    new_seq = []
    feature1 = []
    feature2 = []
    feature3 = []
    feature4 = []
    feature5 = []
    feature6 = []

    i2 = 0
    for i in range(len(aligned_seq)):
        #print(i, aligned_seq[i], ":", i2, raw_seq[i2])
        #print(i, i2, len(raw_seq))
        if aligned_seq[i] == raw_seq[i2]:

            #Append values
            new_seq.append(raw_seq[i2])
            feature1.append(raw_1[i2])
            feature2.append(raw_2[i2])
            feature3.append(raw_3[i2])
            feature4.append(raw_4[i2])
            feature5.append(raw_5[i2])
            feature6.append(raw_6[i2])
            
            i2 += 1
            if i2 >= len(raw_seq):
                break
            #print("Appended")

        else:
            #Append 0
            new_seq.append("-1")
            feature1.append("-1")
            feature2.append("-1")
            feature3.append("-1")
            feature4.append("-1")
            feature5.append("-1")
            feature6.append("-1")
            
    #Final check
    while len(new_seq) < len(aligned_seq):
        new_seq.append("-1")
        feature1.append("-1")
        feature2.append("-1")
        feature3.append("-1")
        feature4.append("-1")
        feature5.append("-1")
        feature6.append("-1")            

            
    #Save to df and append to master df list
    df = pd.DataFrame({"Sequence": new_seq, "SSE": feature1, "CLE": feature2, "ACC": feature3, "pACC": feature4, "CNa": feature5, "CNb": feature6})
    df.index.name = filenames[x]
    dfs.append(df)
    #print("\n\n", new_seq)
    #print("Lengths:", len(aligned_seq), len("".join(raw_seq)), len("".join(new_seq)))

print("Time:", time.time() - start)

Time: 6.654075622558594


In [14]:
len(dfs)

1464

### Append MHC, pep, TCR a/b type per position

#### Structure aligned length calculations:

MHC: (1-181)

GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYEPRARWMEQEG-PEYWERETQKAKGNEQSFRVSLRTLLGYYNQSAGGSHTIQVISGCEVGSDGRLLRGYQQYAYDGCDYIALNEDLKTWTAADMAALITKHKWEQAGEAER-LRAYLEGTCVEWLRRYLKNGNATL

Peptide: (1-11)

RAYGKNSLXXX

TCR: (1-276)

-QSVTQPDARVTVSEGASLQLRCKYSYSATP-------YLFWYVQY-PRQGLQLLLKYYSGD----PVVQ-GVNGFEAEF-S-KSNSSFHLRKASVHWSDSAVYFCAVSGFA-----------SALTFGSGTKVIVLP-AVTQSPRNKVAVTGGKVTLSCNQTNNHN------NMYWYRQDTGHGLRLIHYSYGAGS----TEKG--DIPD-GYKASRPSQENFSLILELATPSQTSVYFCASGGGGT----------------LYFGAGTRLSVL

TCR-A (1-138)

-QSVTQPDARVTVSEGASLQLRCKYSYSATP-------YLFWYVQY-PRQGLQLLLKYYSGD----PVVQ-GVNGFEAEF-S-KSNSSFHLRKASVHWSDSAVYFCAVSGFA-----------SALTFGSGTKVIVLP

TCR-B (1-38)

-AVTQSPRNKVAVTGGKVTLSCNQTNNHN------NMYWYRQDTGHGLRLIHYSYGAGS----TEKG--DIPD-GYKASRPSQENFSLILELATPSQTSVYFCASGGGGT----------------LYFGAGTRLSVL

1-468

0-467 (len(468))

In [17]:
### %time
# Pos 0 - 181 = MHC
# Pos 181 - 192 = Pep
# 192 - 330 = tcr a
# 330 - 468 = tcr b

#Create column of sequence type
dfs[0]["Type"] = 0
dfs[0]["Type"].iloc[0:181] = 1
dfs[0]["Type"].iloc[181:192] = 2
dfs[0]["Type"].iloc[192:330] = 3
dfs[0]["Type"].iloc[330:468] = 4

for i in range(1, len(dfs)):
    dfs[i]["Type"] = dfs[0]["Type"]

/home/maghoi/.conda/envs/fastai_py37/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Validate data

### Save data

In [104]:
!mkdir -p /scratch/maghoi/pMHC_data/features5/
!rm /scratch/maghoi/pMHC_data/features5/*
!ls /scratch/maghoi/pMHC_data/features5/

In [19]:
for i in range(len(dfs)):
    dfs[i].to_csv("/scratch/maghoi/pMHC_data/features5/" + dfs[i].index.name + ".csv")

In [20]:
!ls /scratch/maghoi/pMHC_data/features5/|wc

   1464    1464   45384


In [21]:
len(filenames)

1464

### Biopython

In [97]:
!realpath /home/maghoi/pMHC_data/model_structures/1g6r_1p_N1_1t_80s_pMHC-TCR.pdb

/home/maghoi/pMHC_data/model_structures/1g6r_1p_N1_1t_80s_pMHC-TCR.pdb


In [156]:
test = "GPHSLRYFVTAVSRPGLGEPRYMEVGYVDDTEFVRFDSDAENPRYEPRARWMEQEG-PEYWERETQKAKGNEQSFRVDLRTLLGYYNQSKGGSHTIQVISGCEVGSDGRLLRGYQQYAYDGCDYIALNEDLKTWTAADMAALITKHKWEQAGEAER-LRAYLEGTCVEWLRRYLKNGNATLKNIRYVKVXXX-QSVTQPDARVTVSEGASLQLRCKYSYSATP-------YLFWYVQY-PRQGLQLLLKYYSGD----PVVQ-GVNGFEAEF-S-KSNSSFHLRKASVHWSDSAVYFCAVSGFA-----------SALTFGSGTKVIVLP-AVTQSPRNKVAVTGGKVTLSCNQTNNHN------NMYWYRQDTGHGLRLIHYSYGAGS----TEKG--DIPD-GYKASRPSQENFSLILELATPSQTSVYFCASGGGGT----------------LYFGAGTRLSVL"

In [8]:
the 

,Pos
Rank1,176
Rank2,176


In [4]:
from Bio.PDB import PDBParser

# create parser
parser = PDBParser()

# read structure from file
structure = parser.get_structure('PHA-L', '/home/maghoi/pMHC_data/model_structures/1g6r_1p_N1_1t_80s_pMHC-TCR.pdb')

df = pd.DataFrame(columns = ["4", "6", "8", "10"])

model = structure[0]
chain1 = model['P']
chain2 = model['A']
chain3 = model['B']

len_a = len(chain2)
len_b = len(chain3)

i = 0
# this example uses only the first residue of a single chain.
# it is easy to extend this to multiple chains and residues.
for res1 in chain1:
    for res2, res3 in zip(chain2, chain3):
        if (res1 != res2) and (res1 != res3):
            # compute distance between CA atoms
            try:
                distance1 = res1['CA'] - res2['CA']
                distance2 = res1['CA'] - res3['CA']
            except KeyError:
                ## no CA atom, e.g. for H_NAG
                continue
            if distance1 < 8:
                print(res1, res2, distance1)
            if distance2 < 8:
                pass
                print(res1, res3, distance2)
        # stop after first residue
            #if i > 50:
            #    break

<Residue ASN het=  resseq=2 icode= > <Residue ALA het=  resseq=123 icode= > 7.8525577
<Residue ILE het=  resseq=3 icode= > <Residue PHE het=  resseq=111 icode= > 7.689913
<Residue ILE het=  resseq=3 icode= > <Residue ALA het=  resseq=123 icode= > 5.3024597
<Residue ARG het=  resseq=4 icode= > <Residue PHE het=  resseq=111 icode= > 7.9127197
<Residue ARG het=  resseq=4 icode= > <Residue ALA het=  resseq=123 icode= > 4.774932
<Residue ARG het=  resseq=4 icode= > <Residue SER het=  resseq=124 icode= > 6.334961
<Residue TYR het=  resseq=5 icode= > <Residue GLY het=  resseq=124 icode= > 7.5250535
<Residue TYR het=  resseq=5 icode= > <Residue ALA het=  resseq=123 icode= > 7.846524
<Residue VAL het=  resseq=6 icode= > <Residue GLY het=  resseq=107 icode= > 7.772449
<Residue VAL het=  resseq=6 icode= > <Residue GLY het=  resseq=124 icode= > 3.99298
<Residue VAL het=  resseq=6 icode= > <Residue GLY het=  resseq=125 icode= > 6.064534
<Residue LYS het=  resseq=7 icode= > <Residue ASN het=  resseq

In [ ]:
#Get length chain A
#Get length chain B
# Add up to get aboslute list of residues within 9 Å
# For each residue count proximity to peptide 
# For each residue, count total occurences of binding
# Only for TCR
# For those positions in raw seq:
# Note the relative position
# During feature append, add feature7: value proximity
# Less than 9 = 1
# Less than 7 = 2
# Less than 5 = 3

#https://bioinformatics.stackexchange.com/questions/783/how-can-we-find-the-distance-between-all-residues-in-a-pdb-file

### Load data for pre-proc

In [11]:
import sys
sys.path.append("fastai/old/")
from fastai_v7.structured import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer, OneHotEncoder
import os
import re
import glob
#os.environ["CUDA_VISIBLE_DEVICES"]="7"

from pandas import DataFrame
from IPython.display import HTML

import warnings
warnings.filterwarnings(action='once')
warnings.simplefilter('ignore')


In [12]:
filelist_loaded = []
filelist = glob.glob("/home/maghoi/pMHC_data/features5/*.csv"); len(filelist)
#filelist = glob.glob("/scratch/maghoi/pMHC_data/features5/*.csv"); len(filelist)
for i in range(0, len(filelist)):
    df = pd.read_csv(filelist[i])
    #df = df.drop(["Unnamed: 0", "Num", "pACC", "CNb"], axis = 1)

    #Convert categorical values (res + solvent acc cateogry) to string
    #df.iloc[:, 0:2] = df.iloc[:, 0:2].astype(int).astype(str)
    #print(df.shape)
    filelist_loaded.append(df)

In [13]:
filelist_loaded[0].shape

(468, 9)

In [14]:
filelist_loaded[0].columns[0]

'1g6r_1p_N1_1t_80s_pMHC-TCR'

### Remember to normalize intgerized x

In [77]:
def integerize_ACC(x):
    if x == 0:
        x = 0.05
    if x > 0 and x <= 1:
        x = 0.5
    if x > 1 and x <= 2:
        x = 0.95
    return(x)

In [78]:
def integerize(x):
    if x > 0 and x <= 3:
        x = 0.05
    if x > 3 and x <= 6:
        x = 0.35
    if x > 6 and x <= 9:
        x = 0.65
    if x > 9:
        x = 0.95
    return(x)

#### Energies

#### Add energy data

In [439]:
df_energy = pd.read_feather("main/data/02_Features/Energy_ordered.feather")

In [440]:
df_energy.shape, len(filelist_loaded)

((1464, 60), 1464)

In [377]:
10 % 5

0

In [444]:
rows = list(df_energy.columns[1:])

for i in range(len(filelist_loaded)):
    #check
    if len(filelist_loaded[i]) != 468:
        print(len(filelist_loaded[i], "!=", "468"))
        break
    
    #Construct df to add
    energy_df = df_energy.iloc[i][1:].apply(pd.to_numeric)
    energy_df = pd.DataFrame(np.vstack([np.array(energy_df)]*468))
    
    #add
    filelist_loaded[i][rows] = energy_df
    if i % 100 == 0: print(i)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [435]:
filelist_loaded[0].dtypes

PA_energy Ionisation        float64
PA_electrostatic kon        float64
A_Backbone Hbond            float64
MP_Interaction Energy       float64
A_energy Ionisation         float64
AB_entropy mainchain        float64
PB_entropy sidechain        float64
B_energy Ionisation         float64
PB_Electrostatics           float64
PA_Solvation Hydrophobic    float64
P_entropy sidechain         float64
P_total energy              float64
A_total energy              float64
MP_torsional clash          float64
P_entropy mainchain         float64
MA_helix dipole             float64
PB_Interface Residues         int64
AB_backbone clash           float64
MA_torsional clash          float64
MA_Van der Waals            float64
PA_Interaction Energy       float64
B_entropy sidechain         float64
P_Solvation Polar           float64
MA_Solvation Polar          float64
MA_entropy sidechain        float64
MP_Solvation Polar          float64
MP_IntraclashesGroup2       float64
MB_Interface Residues       

In [446]:
backup = filelist_loaded.copy()

#### Save data

In [447]:
import pickle
with open("/scratch/maghoi/pMHC_data/02_Features/filelist_loaded.pickle", "wb") as fp:
    pickle.dump(filelist_loaded, fp)

#### Load data

In [393]:
with open("/scratch/maghoi/pMHC_data/02_Features/filelist_loaded.pickle", "rb") as fp:
    filelist_loaded = pickle.load(fp)

### 1-hot encode and integerize CNa , Cnb + position

In [448]:
for i in range(0, len(filelist_loaded)):
    #Set continuous position
    filelist_loaded[i].iloc[0:468, 0] = filelist_loaded[i].iloc[0:468, 0].apply(lambda x: (x+1) / 489)

for i in range(0, len(filelist_loaded)):
    #Fix position vector:
    
    #Turn continous numbers for CNa and CNb to categorical numbers for later 1-hot
    filelist_loaded[i].loc[:, "CNa"] = filelist_loaded[i].loc[:, "CNa"].apply(lambda x: integerize(x))
    filelist_loaded[i].loc[:, "CNb"] = filelist_loaded[i].loc[:, "CNb"].apply(lambda x: integerize(x))
    filelist_loaded[i].loc[:, "ACC"] = filelist_loaded[i].loc[:, "ACC"].apply(lambda x: integerize_ACC(x))
    
    #Set position number per protein type
    filelist_loaded[i].iloc[0:181, 0] = filelist_loaded[i].iloc[0:181, 0].apply(lambda x: (x+1) / 182)
    filelist_loaded[i].iloc[181:192, 0] = filelist_loaded[i].iloc[181:192, 0].apply(lambda x: (x+1-181) / 12)
    filelist_loaded[i].iloc[192:, 0] = filelist_loaded[i].iloc[192:, 0].apply(lambda x: (x+1-192) / 277)
    
    #Set missing values to 0 (should have done this beforehand)
    filelist_loaded[i].loc[:, "CNa"] = filelist_loaded[i].loc[:, "CNa"].apply(lambda x: np.where(x == -1, 0, x))
    filelist_loaded[i].loc[:, "CNb"] = filelist_loaded[i].loc[:, "CNb"].apply(lambda x: np.where(x == -1, 0, x))
    filelist_loaded[i].loc[:, "ACC"] = filelist_loaded[i].loc[:, "ACC"].apply(lambda x: np.where(x == -1, 0, x))
    #filelist_loaded[i].loc[:, "pACC"] = filelist_loaded[i].loc[:, "pACC"].apply(lambda x: np.where(x == -1, 0, x))
    filelist_loaded[i] = filelist_loaded[i].drop("pACC", axis = 1)

In [449]:
HTML(pd.DataFrame(filelist_loaded[0][180:195]).to_html())

,1g6r_1p_N1_1t_80s_pMHC-TCR,Sequence,SSE,CLE,ACC,pACC,CNa,CNb,Type,PA_energy Ionisation,PA_electrostatic kon,A_Backbone Hbond,MP_Interaction Energy,A_energy Ionisation,AB_entropy mainchain,PB_entropy sidechain,B_energy Ionisation,PB_Electrostatics,PA_Solvation Hydrophobic,P_entropy sidechain,P_total energy,A_total energy,MP_torsional clash,P_entropy mainchain,MA_helix dipole,PB_Interface Residues,AB_backbone clash,MA_torsional clash,MA_Van der Waals,PA_Interaction Energy,B_entropy sidechain,P_Solvation Polar,MA_Solvation Polar,MA_entropy sidechain,MP_Solvation Polar,MP_IntraclashesGroup2,MB_Interface Residues,MA_Interaction Energy,AB_entropy sidechain,M_backbone clash,PA_Electrostatics,MP_electrostatic kon,AB_Sidechain Hbond,P_backbone clash,A_entropy sidechain,AB_Electrostatics,M_disulfide,MP_Electrostatics,MB_torsional clash,MP_Backbone Hbond,B_torsional clash,MB_energy Ionisation,MA_Electrostatics,A_Electrostatics,MA_Van der Waals clashes,AB_Solvation Polar,PA_entropy mainchain,B_disulfide,A_backbone clash,MB_electrostatic kon,PA_torsional clash,MP_Sidechain Hbond,B_Sidechain Hbond,A_Solvation Hydrophobic,A_Sidechain Hbond,B_entropy mainchain,P_Solvation Hydrophobic,PB_Van der Waals clashes
180,0.370143,L,L,R,1,23,5,8,1,0.0,0.016093,-61.7341,-10.693,0.115684,4.51076,3.44592,0.275566,-0.688443,-1.89969,2.01793,10.3272,78.5875,2.04372,5.78208,-0.236418,7.0,0.868306,1.28293,-6.63675,2.6506,56.7949,5.24075,8.72135,3.13317,34.0057,4.15582,23.0,1.79638,3.12196,143.654,0.036407,-0.467085,-3.03614,0.0,57.3258,0.087374,-4.25849,-5.28926,0.19795,-5.24609,8.73494,5.551120e-17,-0.588257,-0.545424,6.76937,11.4988,1.19555,-4.43037,62.9279,-0.045508,0.054021,-8.8985,-26.9186,-163.314,-21.5245,155.303,-2.66971,2.2648
181,0.372188,K,L,R,0,0,4,9,2,0.0,0.016093,-61.7341,-10.693,0.115684,4.51076,3.44592,0.275566,-0.688443,-1.89969,2.01793,10.3272,78.5875,2.04372,5.78208,-0.236418,7.0,0.868306,1.28293,-6.63675,2.6506,56.7949,5.24075,8.72135,3.13317,34.0057,4.15582,23.0,1.79638,3.12196,143.654,0.036407,-0.467085,-3.03614,0.0,57.3258,0.087374,-4.25849,-5.28926,0.19795,-5.24609,8.73494,5.551120e-17,-0.588257,-0.545424,6.76937,11.4988,1.19555,-4.43037,62.9279,-0.045508,0.054021,-8.8985,-26.9186,-163.314,-21.5245,155.303,-2.66971,2.2648
182,0.374233,N,L,R,0,0,5,10,2,0.0,0.016093,-61.7341,-10.693,0.115684,4.51076,3.44592,0.275566,-0.688443,-1.89969,2.01793,10.3272,78.5875,2.04372,5.78208,-0.236418,7.0,0.868306,1.28293,-6.63675,2.6506,56.7949,5.24075,8.72135,3.13317,34.0057,4.15582,23.0,1.79638,3.12196,143.654,0.036407,-0.467085,-3.03614,0.0,57.3258,0.087374,-4.25849,-5.28926,0.19795,-5.24609,8.73494,5.551120e-17,-0.588257,-0.545424,6.76937,11.4988,1.19555,-4.43037,62.9279,-0.045508,0.054021,-8.8985,-26.9186,-163.314,-21.5245,155.303,-2.66971,2.2648
183,0.376278,I,L,C,0,0,3,7,2,0.0,0.016093,-61.7341,-10.693,0.115684,4.51076,3.44592,0.275566,-0.688443,-1.89969,2.01793,10.3272,78.5875,2.04372,5.78208,-0.236418,7.0,0.868306,1.28293,-6.63675,2.6506,56.7949,5.24075,8.72135,3.13317,34.0057,4.15582,23.0,1.79638,3.12196,143.654,0.036407,-0.467085,-3.03614,0.0,57.3258,0.087374,-4.25849,-5.28926,0.19795,-5.24609,8.73494,5.551120e-17,-0.588257,-0.545424,6.76937,11.4988,1.19555,-4.43037,62.9279,-0.045508,0.054021,-8.8985,-26.9186,-163.314,-21.5245,155.303,-2.66971,2.2648
184,0.378323,R,L,C,0,2,7,7,2,0.0,0.016093,-61.7341,-10.693,0.115684,4.51076,3.44592,0.275566,-0.688443,-1.89969,2.01793,10.3272,78.5875,2.04372,5.78208,-0.236418,7.0,0.868306,1.28293,-6.63675,2.6506,56.7949,5.24075,8.72135,3.13317,34.0057,4.15582,23.0,1.79638,3.12196,143.654,0.036407,-0.467085,-3.03614,0.0,57.3258,0.087374,-4.25849,-5.28926,0.19795,-5.24609,8.73494,5.551120e-17,-0.588257,-0.545424,6.76937,11.4988,1.19555,-4.43037,62.9279,-0.045508,0.054021,-8.8985,-26.9186,-163.314,-21.5245,155.303,-2.66971,2.2648
185,0.380368,Y,L,C,0,0,4,6,2,0.0,0.016093,-61.7341,-10.693,0.115684,4.51076,3.44592,0.275566,-0.688443,-1.89969,2.01793,10.3272,78.5875,2.04372,5.78208,-0.236418,7.0,0.868306,1.28293,-6

#### small analysis (...)

In [14]:
df = filenames.iloc[:, 0]
pos_idx = df.str.contains(".*P1.*").get_values()
neg_idx = df.str.contains(".*N1.*").get_values()
switch_idx = df.str.contains(".*S1.*").get_values()


pos_idx = filenames[pos_idx].index.get_values()
neg_idx = filenames[neg_idx].index.get_values()
switch_idx = filenames[switch_idx].index.get_values()

NameError: name 'filenames' is not defined

In [ ]:
pos = pd.DataFrame()
for i in pos_idx:
    df = filelist_loaded[i].loc[181:192, "CNa"]
    pos = pos.append(df)
    
neg = pd.DataFrame()
for i in neg_idx:
    df = filelist_loaded[i].loc[181:192, "CNa"]
    neg = neg.append(df)
    
switch = pd.DataFrame()
for i in switch_idx:
    df = filelist_loaded[i].loc[181:192, "CNa"]
    switch = switch.append(df)

In [ ]:
from scipy.stats import mannwhitneyu

for i in range(181, 192):
    print("\nPosition", i)
    print("Pos vs neg:", scipy.stats.mannwhitneyu(np.array(pos.loc[:, i]).flatten(), np.array(neg.loc[:, i]).flatten())[1] )
    print("Pos vs switch:", scipy.stats.mannwhitneyu(np.array(pos.loc[:, i]).flatten(), np.array(switch.loc[:, i]).flatten())[1] )
    #print("Neg vs switch:", scipy.stats.mannwhitneyu(np.array(neg.loc[:, i]).flatten(), np.array(switch.loc[:, i]).flatten()))[1]
    
    

In [ ]:
Connections:
0: 0-2
1: 3-5
2: 6-8
3: 9+

### 1-hot dummy rows

In [ ]:
rows = list(df_energy.columns[1:])

for i in range(len(filelist_loaded)):
    #check
    if len(filelist_loaded[i]) != 468:
        print(len(filelist_loaded[i], "!=", "468"))
        break

    #Construct df to add
    energy_df = df_energy.iloc[i][1:]
    energy_df = pd.DataFrame(np.vstack([np.array(energy_df)]*468))

    #add
    filelist_loaded[i][rows] = energy_df
    if i % 100 == 0: print(i)

In [469]:
len(rows)

59

In [471]:
rows = list(df_energy.columns[1:])
dummy[rows] = pd.DataFrame(np.zeros(shape = (len(dummy), len(rows))))

In [671]:
dummy = pd.DataFrame({"Position": np.tile(range(1,2), 21)[0:21],
            "Sequence": range(0,21),
            "SSE": np.tile(range(0,9), 6)[0:21],
            "CLE": np.tile(range(0,20), 6)[0:21],
            "ACC": np.tile(range(0,4), 21)[0:21],
            "pACC": np.tile(range(0,1), 21)[0:21], #disregard
            "CNa": np.tile(range(0,1), 21)[0:21], #disregard
            "CNb": np.tile(range(0,1), 21)[0:21], #disregard
            "Type": np.tile(range(1,5), 7)[0:21]})
#            "pACC": np.tile(range(0,2), 21)[0:21],
#add energy rows
rows = list(df_energy.columns[1:])
dummy[rows] = pd.DataFrame(np.zeros(shape = (len(dummy), len(rows))))
dummy

,Position,Sequence,SSE,CLE,ACC,pACC,CNa,CNb,Type,PA_energy Ionisation,...,A_backbone clash,MB_electrostatic kon,PA_torsional clash,MP_Sidechain Hbond,B_Sidechain Hbond,A_Solvation Hydrophobic,A_Sidechain Hbond,B_entropy mainchain,P_Solvation Hydrophobic,PB_Van der Waals clashes
0,1,0,0,0,0,0,0,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,1,1,0,0,0,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,2,2,2,0,0,0,3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,3,3,3,0,0,0,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,4,4,0,0,0,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,5,5,5,1,0,0,0,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,6,6,6,2,0,0,0,3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,7,7,7,3,0,0,0,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,8,8,8,0,0,0,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,9,0,9,1,0,0,0,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1-hot, normalize

In [684]:
dfs = []
for i in range(len(filelist_loaded[:])):
    #Process categorical to values using fast.ai proc_df (only processes string values)
    df = proc_df(filelist_loaded[i])[0] 
    if len(df) != 468:
        print("Expected df length", 468, "found", len(df))
        break

    #Change proc_df´ed values to 0 = missing value, 1 as lowest real value
    #Change rest of features to 0 = missing value, 1 as lowest real value
    #df.iloc[:, [1, 2, 3]] = df.iloc[:, [1, 2, 3]] - 1
    #df.iloc[:, [4, 5, 6, 7]] = df.iloc[:, [4, 5, 6, 7]] + 1

    #Append dummy column
    df.columns = dummy.columns.values
    
    #Remove -1 values
    df.iloc[:, [4,5,6,7]] += 1
    df = df.append(dummy)

    #One-hot encoding for Sequence, SSE, CLE, ACC, type
    onehotencoder = OneHotEncoder(categorical_features = [1,2,3,4,8])
    df = onehotencoder.fit_transform(df).toarray()
    
    #Remove 1-hot columns encoding missing positions where value = 1 (unnecessary)
    #Columns correspond to first columns for seq, SSE, CLE, ACC
    #Remove dummy columns
    #df = np.delete(df, [0], axis = 1)
    df = df[0:468]
    
    #Normalization for columns sequence position, pACC, CNa, CNb
    #df[:, -4: ] = preprocessing.normalize(df[:, -4: ])
    
    dfs.append(df)
    
    if df.shape != (468, 122):
        print(i, "expected shape", (468, 122), "got", df.shape)
        break

    if i % 100 == 0: print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [687]:
for i in range(len(dfs)):
    filename = filelist_loaded[i].columns[0]
    np.save("/scratch/maghoi/pMHC_data/features10/" + filename, dfs[i])

In [ ]:
for i in range(len(dfs)):
    filename = filelist_loaded[i].columns[0]
    np.save("/scratch/maghoi/pMHC_data/features10/" + filename, dfs[i])

#### Load old56, add energy features normalized, save in features12

In [133]:
p0 = (glob.glob("/home/maghoi/pMHC_data/features6/*0p*"))
p1 = (glob.glob("/home/maghoi/pMHC_data/features6/*1p*"))
p2 = (glob.glob("/home/maghoi/pMHC_data/features6/*2p*"))
p3 = (glob.glob("/home/maghoi/pMHC_data/features6/*3p*"))
p4 = (glob.glob("/home/maghoi/pMHC_data/features6/*4p*"))

In [3]:
np.load(glob.glob("/home/maghoi/pMHC_data/features12/*0p*")[0]).shape

(468, 115)

In [76]:
# Find and append all energies from complex names per partition
nps_energy = []
for p in [p0, p1, p2, p3, p4]:
    idxs = []
    for i in range(len(p)):
        name = p0[i][33:][:-4]
        energy_idx = df_energy[df_energy.iloc[:, 0] == name].index[0]
        idxs.append(energy_idx)
    
    np_energy = np.array(df_energy.iloc[:, 1:].iloc[idxs])
    nps_energy.append(np_energy)

In [77]:
# stack and normalize nps energy
X = np.vstack((nps_energy[0:5]))

from sklearn import preprocessing
X_normalized = preprocessing.normalize(X, norm='l2')

In [145]:
pd.DataFrame(X_normalized).describe()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
count,1.464000e+03,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,...,1464.000000,1464.000000,1.464000e+03,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1464.000000,1.464000e+03
mean,1.249982e-07,0.000045,-0.214212,-0.038263,0.000351,0.013288,0.004820,0.000818,0.000029,-0.010606,...,0.244475,-0.000831,9.318937e-04,-0.024020,-0.091380,-0.490056,-0.084054,0.538954,-0.009438,1.243080e-02
std,1.686894e-06,0.000372,0.018747,0.019334,0.000497,0.003717,0.002740,0.000587,0.000796,0.005039,...,0.023924,0.000601,1.523824e-03,0.006191,0.013615,0.015590,0.010960,0.014940,0.004503,1.495249e-02
min,0.000000e+00,-0.000806,-0.252197,-0.072797,0.000000,0.005660,0.000023,0.000155,-0.004129,-0.031386,...,0.196237,-0.002295,-1.152711e-17,-0.047718,-0.126647,-0.527117,-0.111563,0.482021,-0.020282,-1.210071e-17
25%,0.000000e+00,-0.000131,-0.227549,-0.052121,0.000000,0.010339,0.002637,0.000196,-0.000044,-0.013119,...,0.227164,-0.001244,5.946017e-05,-0.027433,-0.101904,-0.500425,-0.091202,0.529431,-0.012283,1.372381e-03
50%,0.000000e+00,0.000000,-0.218403,-0.042167,0.000240,0.012974,0.004261,0.000937,0.000132,-0.009620,...,0.239265,-0.000805,3.156360e-04,-0.023200,-0.091520,-0.491952,-0.083170,0.541626,-0.008186,6.488329e-03
75%,0.000000e+00,0.000191,-0.201047,-0.026823,0.000487,0.016071,0.006755,0.001063,0.000344,-0.007010,...,0.257593,-0.000380,1.272130e-03,-0.019767,-0.081138,-0.480821,-0.076735,0.549649,-0.005693,1.825334e-02
max,2.287467e-05,0.001562,-0.139568,0.069503,0.003657,0.023814,0.013435,0.003548,0.003548,-0.001534,...,0.345020,0.000570,1.014730e-02,-0.009697,-0.055361,-0.414609,-0.052807,0.574612,-0.002266,7.405981e-02


In [102]:
energy_df = X[1]
energy_df = pd.DataFrame(np.vstack([np.array(energy_df)]*468))

In [105]:
np.hstack((np.load(p0[0]), energy_df)).shape

(468, 115)

In [146]:
filelist = p0+p1+p2+p3+p4
filelist_new = []
names = []

for i in range(len(filelist)):
    name = filelist[i][33:][:-4]
    names.append(name)
    #check
    df = np.load(filelist[i])
    
    if len(df) != 468:
        print(len(filelist_loaded[i], "!=", "468"))
        break

    #Construct df to add
    energy_df = X_normalized[i]
    energy_df = pd.DataFrame(np.vstack([np.array(energy_df)]*468))
    df = np.hstack((df, energy_df))

    #add
    filelist_new.append(df)
    if i % 100 == 0: print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


#### Save features56 + all energies

In [150]:
!mkdir pMHC_data/02_Features/Features12/

In [152]:
for name, df in zip(names, filelist_new):
    filename = "pMHC_data/02_Features/Features12/" + name + ".npy"
    np.save(filename, df)

In [ ]:
# done

In [97]:
parts = [0, p0, p1, p2, p3, p4]

for i in range(len([0, p0, p1, p2, p3, p4])):
    start = len(parts[0:i])
    end = len(parts[i])

0
1
2
3
4
5


In [85]:
#Extract normalized entries and put back into partitions
for p in [p0]:
    for i in range(len(p[0:2])):
        test = np.load(p[i])

AttributeError: 'numpy.ndarray' object has no attribute 'load'

In [92]:
np.load((p[0]))

array([[0.     , 0.     , 0.     , ..., 0.99907, 0.02968, 0.02968],
       [0.     , 0.     , 0.     , ..., 0.99085, 0.09289, 0.09289],
       [0.     , 0.     , 0.     , ..., 0.78621, 0.3669 , 0.47173],
       ...,
       [0.     , 0.     , 0.     , ..., 0.09398, 0.01709, 0.01068],
       [0.     , 0.     , 0.     , ..., 0.05769, 0.0235 , 0.0235 ],
       [0.     , 0.     , 0.     , ..., 0.     , 0.     , 0.     ]])

In [79]:
X_normalized[0]

array([ 0.     , -0.00021, -0.20825, -0.03536,  0.     ,  0.01255,  0.00675,  0.00097, -0.00049, -0.0083 ,
        0.01456,  0.02546,  0.17739,  0.0042 ,  0.02587,  0.00061,  0.05709,  0.00486,  0.00116, -0.01532,
        0.00094,  0.18828,  0.02672,  0.02195,  0.0094 ,  0.09215,  0.00743,  0.09739, -0.00464,  0.01922,
        0.44304, -0.00039,  0.00033, -0.01395,  0.00607,  0.20579, -0.00225, -0.01437, -0.00632,  0.00211,
       -0.0162 ,  0.03727, -0.     , -0.00212, -0.01017,  0.00206,  0.04998,  0.00189, -0.01482,  0.28327,
       -0.00046,  0.     , -0.01952, -0.09352, -0.51211, -0.08452,  0.5105 , -0.01877,  0.00955])

In [ ]:
#Normalize entire array for 


i

### Create numpy array, normalize, then save again

In [695]:

def numpy_array(datalist):
    data_size = len(datalist)

    #Initialize empty df ordered by complexes and aminos
    dim1 = range(0, data_size)
    dim2 = datalist[0].shape[0]
    dim3 = datalist[0].shape[1]
    x = np.zeros(shape = (data_size, dim2, dim3))

    for i in range(0, data_size):
        x[i] = datalist[i]
        
        if i % 100 == 0: print(i)
        
    return x

In [711]:
X.shape

(1464, 468, 122)

In [712]:
X0 = X[:, :, 0:21]
X1 = X[:, :, 21:]

In [713]:
#Normalize entire array for 
train_data = X1

import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_instances, num_time_steps, num_features = train_data.shape
train_data = np.reshape(train_data, newshape=(-1, num_features))
train_data = scaler.fit_transform(train_data)

train_data = np.reshape(train_data, newshape=(num_instances, num_time_steps, num_features))



In [ ]:
X = np.dstack((X0, X1)).shape

In [727]:
!mkdir -p /scratch/maghoi/pMHC_data/features11/

In [728]:
dfs = X
for i in range(len(dfs)):
    filename = filelist_loaded[i].columns[0]
    np.save("/scratch/maghoi/pMHC_data/features11/" + filename, dfs[i])

In [688]:
!ls -al /scratch/maghoi/pMHC_data/features10/|wc

   1467   13196  118702


In [689]:
!du -h /scratch/maghoi/pMHC_data/features10/

641M	/scratch/maghoi/pMHC_data/features10/


In [562]:
np.load("/scratch/maghoi/pMHC_data/features7/5c0b_1p_N1_1t_95s_pMHC-TCR.npy")

array([[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.00549, 101.     ],
       [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.01099,  69.     ],
       [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.01648,  17.     ],
       ...,
       [  0.     ,   0.     ,   0.     , ...,   1.     ,   0.98917,  28.     ],
       [  0.     ,   0.     ,   0.     , ...,   1.     ,   0.99278,   1.     ],
       [  0.     ,   0.     ,   0.     , ...,   1.     ,   0.99639,  77.     ]])

In [563]:
np.load("/scratch/maghoi/pMHC_data/features7/5men_0p_P1_1t_90s_pMHC-TCR.npy")

array([[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.00549, 101.     ],
       [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.01099,  67.     ],
       [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.01648,  13.     ],
       ...,
       [  0.     ,   0.     ,   0.     , ...,   1.     ,   0.98917,  48.     ],
       [  0.     ,   0.     ,   0.     , ...,   1.     ,   0.99278,   7.     ],
       [  0.     ,   0.     ,   0.     , ...,   1.     ,   0.99639,  71.     ]])

#### Add on energy terms

In [290]:
df_energy = pd.read_csv("/home/maghoi/main/data/All_Energy.csv")

In [291]:
df_energy["M_total energy"]

0       43.8599
1       33.0047
2       32.7762
3       25.6722
4       26.2620
5       42.5881
6       47.9869
7       35.6672
8       43.3983
9       34.7800
10      20.7450
11      42.5881
12      38.0777
13      32.0869
14      36.0466
15      23.7637
16      21.3380
17      42.5881
18      61.4802
19      32.6174
20      24.0108
21      29.6723
22      19.5764
23      46.4203
24      59.4287
25      36.6789
26      28.8329
27      39.6564
28      41.5657
29      63.7752
         ...   
1434    43.1022
1435    26.9029
1436    20.4134
1437    32.6884
1438    23.0201
1439    56.5885
1440    34.3450
1441    43.5015
1442    40.9215
1443    36.6938
1444    37.4091
1445    57.1499
1446    31.3410
1447    42.8556
1448    35.5864
1449    37.0827
1450    33.9896
1451    28.7505
1452    41.5823
1453    32.3616
1454    22.2436
1455    37.7567
1456    31.3661
1457    27.1967
1458    40.4614
1459    36.9587
1460    31.5761
1461    40.1811
1462    36.1609
1463    42.2487
Name: M_total energy, Le

Done!

In [293]:
ordered = ["Unnamed: 0",
"PA_energy Ionisation",
"PA_electrostatic kon",
"A_Backbone Hbond",
"MP_Interaction Energy",
"A_energy Ionisation",
"AB_entropy mainchain",
"PB_entropy sidechain",
"B_energy Ionisation",
"PB_Electrostatics",
"PA_Solvation Hydrophobic",
"P_entropy sidechain",
"P_total energy",
"A_total energy",
"MP_torsional clash",
"P_entropy mainchain",
"MA_helix dipole",
"PB_Interface Residues",
"AB_backbone clash",
"MA_torsional clash",
"MA_Van der Waals",
"PA_Interaction Energy",
"B_entropy sidechain",
"P_Solvation Polar",
"MA_Solvation Polar",
"MA_entropy sidechain",
"MP_Solvation Polar",
"MP_IntraclashesGroup2",
"MB_Interface Residues",
"MA_Interaction Energy", 
"AB_entropy sidechain",
"M_backbone clash",
"PA_Electrostatics",
"MP_electrostatic kon",
"AB_Sidechain Hbond",
"P_backbone clash",
"A_entropy sidechain",
"AB_Electrostatics",
"M_disulfide",
"MP_Electrostatics",
"MB_torsional clash",
"MP_Backbone Hbond",
"B_torsional clash",
"MB_energy Ionisation",
"MA_Electrostatics",
"A_Electrostatics",
"MA_Van der Waals clashes",
"AB_Solvation Polar",
"PA_entropy mainchain",
"B_disulfide",
"A_backbone clash",
"MB_electrostatic kon",
"PA_torsional clash",
"MP_Sidechain Hbond",
"B_Sidechain Hbond",
"A_Solvation Hydrophobic",
"A_Sidechain Hbond",
"B_entropy mainchain",
"P_Solvation Hydrophobic",
"PB_Van der Waals clashes"]

In [294]:
df_energy.columns

Index(['Unnamed: 0', 'M_total energy', 'M_Backbone Hbond', 'M_Sidechain Hbond',
       'M_Van der Waals', 'M_Electrostatics', 'M_Solvation Polar',
       'M_Solvation Hydrophobic', 'M_Van der Waals clashes',
       'M_entropy sidechain',
       ...
       'AB_Entropy Complex', 'AB_Number of Residues', 'AB_Interface Residues',
       'AB_Interface Residues Clashing', 'AB_Interface Residues VdW Clashing',
       'AB_Interface Residues BB Clashing', 'M_intface', 'P_intface',
       'A_intface', 'B_intface'],
      dtype='object', length=267)

In [295]:
ordered = pd.Series(ordered)
df_energy = df_energy[ordered]

In [296]:
df_energy.shape

(1464, 60)

In [5]:
#extract order of two dfs
namelist = []
for i in range(len(filelist_loaded)):
    namelist.append(filelist_loaded[i].columns[0])
namelist = pd.Series(namelist)

energy_namelist = []
for i in range(len(df_energy)):
    energy_namelist.append(df_energy.iloc[i,0])
energy_namelist = pd.Series(energy_namelist)

print(len(namelist), len(energy_namelist))

master_dict = {}
for i in range(len(energy_namelist)):
    master_dict[i] = energy_namelist[energy_namelist == namelist[i]].index[0]

NameError: name 'df_energy' is not defined

In [299]:
df_energy2 = df_energy.copy()

#Create master_index, set and reset new df
master_index = pd.Series(list(master_dict.values()))
df_energy2 = df_energy2.iloc[master_index]
df_energy2.reset_index(inplace = True, drop = True)

#Check if correct
print(sum(df_energy2.iloc[:, 0] == namelist), len(namelist))

1464 1464


In [300]:
df_energy2.to_feather("main/data/02_Features/Energy_ordered.feather")

In [73]:
df_energy = pd.read_feather("main/data/02_Features/Energy_ordered.feather")

In [74]:
df_energy

,Unnamed: 0,PA_energy Ionisation,PA_electrostatic kon,A_Backbone Hbond,MP_Interaction Energy,A_energy Ionisation,AB_entropy mainchain,PB_entropy sidechain,B_energy Ionisation,PB_Electrostatics,...,A_backbone clash,MB_electrostatic kon,PA_torsional clash,MP_Sidechain Hbond,B_Sidechain Hbond,A_Solvation Hydrophobic,A_Sidechain Hbond,B_entropy mainchain,P_Solvation Hydrophobic,PB_Van der Waals clashes
0,1g6r_1p_N1_1t_80s_pMHC-TCR,0.000000e+00,0.016093,-61.7341,-10.69300,0.115684,4.51076,3.445920,0.275566,-0.688443,...,62.9279,-0.045508,5.402050e-02,-8.89850,-26.9186,-163.314,-21.5245,155.303,-2.669710,2.264800e+00
1,3tjh_4p_P1_1t_90s_pMHC-TCR,0.000000e+00,-0.169053,-63.5736,-7.52432,0.304471,4.06289,1.538010,0.856211,-0.137794,...,72.8137,-0.055644,3.932720e-03,-5.96079,-30.5835,-164.351,-22.5547,155.788,-1.431770,4.678590e+00
2,2bnq_2p_P1_1t_99s_pMHC-TCR,0.000000e+00,0.000000,-69.4683,-11.25950,0.048782,1.96981,2.176420,0.294327,0.063702,...,94.9006,-0.392534,2.272890e-01,-5.04822,-24.0878,-154.845,-35.8666,157.196,-2.660260,1.526360e-01
3,1g6r_1p_N1_1t_90s_pMHC-TCR,0.000000e+00,0.059112,-65.8781,-16.82350,0.307207,5.92017,1.136940,0.268039,0.206121,...,56.9811,-0.045866,5.157470e-02,-9.76213,-27.5204,-157.676,-21.6588,149.108,-2.731350,6.027820e-02
4,1g6r_1p_N1_1t_95s_pMHC-TCR,0.000000e+00,0.038986,-66.2646,-12.34210,0.120750,4.37077,2.909650,0.268053,-0.157241,...,55.3946,-0.055325,3.001690e-05,-7.86236,-27.1086,-160.748,-21.5001,153.647,-3.018520,3.712460e-01
5,1mwa_1p_N3_1t_99s_pMHC-TCR,0.000000e+00,0.110072,-56.6332,-11.02930,0.307053,4.71365,0.903283,0.270716,-0.086970,...,71.8147,-0.060888,3.263490e-04,-7.67914,-28.9789,-157.355,-22.8460,155.648,-1.528870,6.942830e-01
6,1mwa_1p_P1_1t_80s_pMHC-TCR,0.000000e+00,-0.024235,-53.8183,-5.74689,0.130066,4.55771,2.470940,0.245255,-0.007061,...,61.3022,0.065141,3.259190e-01,-8.44047,-24.3638,-159.431,-24.3916,155.703,-2.777930,7.681940e+00
7,1mwa_1p_P1_1t_90s_pMHC-TCR,0.000000e+00,-0.059522,-58.8736,-8.34524,0.103935,3.90352,0.669740,0.293456,-0.100716,...,61.0798,0.019882,1.173310e-04,-7.47632,-23.8989,-157.596,-19.1253,151.936,-2.318600,2.513510e+00
8,1mwa_1p_P1_1t_95s_pMHC-TCR,0.000000e+00,-0.027598,-61.2884,-8.05019,0.309257,4.20270,0.168438,0.328447,-0.105158,...,59.9084,0.076854,1.011110e-02,-6.53870,-26.9531,-160.229,-21.4730,156.400,-2.651270,2.276620e+00
9,1mwa_1p_P1_1t_99s_pMHC-TCR,0.000000e+00,-0.021149,-54.6212,-12.48150,0.122120,4.08154,0.377486,0.310914,-0.103637,...,70.0754,0.125175,4.277190e-04,-9.90183,-29.2920,-155.516,-19.3766,156.163,-2.523270,8.747250e-01
